## 准备数据

In [2]:
import os
import numpy as np
import torch
from torchvision import datasets, transforms

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])

    train_dataset = datasets.MNIST(root='./data', train=True,
                                   download=True, transform=transform)
    x_train = train_dataset.data.float() / 255.0
    y_train = train_dataset.targets

    test_dataset = datasets.MNIST(root='./data', train=False,
                                  download=True, transform=transform)
    x_test = test_dataset.data.float() / 255.0
    y_test = test_dataset.targets

    return (x_train, y_train), (x_test, y_test)

## Demo numpy based auto differentiation

In [3]:
import numpy as np

class Matmul:
    def __init__(self):
        self.mem = {}
        
    def forward(self, x, W):
        h = np.matmul(x, W)
        self.mem={'x': x, 'W':W}
        return h
    
    def backward(self, grad_y):
        '''
        x: shape(N, d)
        w: shape(d, d')
        grad_y: shape(N, d')
        '''
        x = self.mem['x']
        W = self.mem['W']
        
        ####################
        '''计算矩阵乘法的对应的梯度'''
        ####################
        grad_x = np.matmul(grad_y, W.T)
        grad_W = np.matmul(x.T, grad_y)
        return grad_x, grad_W


class Relu:
    def __init__(self):
        self.mem = {}
        
    def forward(self, x):
        self.mem['x']=x
        return np.where(x > 0, x, np.zeros_like(x))
    
    def backward(self, grad_y):
        '''
        grad_y: same shape as x
        '''
        ####################
        '''计算relu 激活函数对应的梯度'''
        ####################
        x = self.mem['x']
        grad_x = grad_y * (x > 0).astype(np.float32)
        return grad_x
    


class Softmax:
    '''
    softmax over last dimention
    '''
    def __init__(self):
        self.epsilon = 1e-12
        self.mem = {}
        
    def forward(self, x):
        '''
        x: shape(N, c)
        '''
        x_exp = np.exp(x)
        partition = np.sum(x_exp, axis=1, keepdims=True)
        out = x_exp/(partition+self.epsilon)
        
        self.mem['out'] = out
        self.mem['x_exp'] = x_exp
        return out
    
    def backward(self, grad_y):
        '''
        grad_y: same shape as x
        '''
        s = self.mem['out']
        sisj = np.matmul(np.expand_dims(s,axis=2), np.expand_dims(s, axis=1)) # (N, c, c)
        g_y_exp = np.expand_dims(grad_y, axis=1)
        tmp = np.matmul(g_y_exp, sisj) #(N, 1, c)
        tmp = np.squeeze(tmp, axis=1)
        tmp = -tmp+grad_y*s 
        return tmp
    
class Log:
    '''
    softmax over last dimention
    '''
    def __init__(self):
        self.epsilon = 1e-12
        self.mem = {}
        
    def forward(self, x):
        '''
        x: shape(N, c)
        '''
        out = np.log(x+self.epsilon)
        
        self.mem['x'] = x
        return out
    
    def backward(self, grad_y):
        '''
        grad_y: same shape as x
        '''
        x = self.mem['x']
        
        return 1./(x+1e-12) * grad_y
    


## Gradient check

In [24]:
# import tensorflow as tf

# x = np.random.normal(size=[5, 6])
# W = np.random.normal(size=[6, 4])
# aa = Matmul()
# out = aa.forward(x, W) # shape(5, 4)
# grad = aa.backward(np.ones_like(out))
# print (grad)

# with tf.GradientTape() as tape:
#     x, W = tf.constant(x), tf.constant(W)
#     tape.watch(x)
#     y = tf.matmul(x, W)
#     loss = tf.reduce_sum(y)
#     grads = tape.gradient(loss, x)
#     print (grads)

# import tensorflow as tf

# x = np.random.normal(size=[5, 6])
# aa = Relu()
# out = aa.forward(x) # shape(5, 4)
# grad = aa.backward(np.ones_like(out))
# print (grad)

# with tf.GradientTape() as tape:
#     x= tf.constant(x)
#     tape.watch(x)
#     y = tf.nn.relu(x)
#     loss = tf.reduce_sum(y)
#     grads = tape.gradient(loss, x)
#     print (grads)

# import tensorflow as tf
# x = np.random.normal(size=[5, 6], scale=5.0, loc=1)
# label = np.zeros_like(x)
# label[0, 1]=1.
# label[1, 0]=1
# label[1, 1]=1
# label[2, 3]=1
# label[3, 5]=1
# label[4, 0]=1
# print(label)
# aa = Softmax()
# out = aa.forward(x) # shape(5, 6)
# grad = aa.backward(label)
# print (grad)

# with tf.GradientTape() as tape:
#     x= tf.constant(x)
#     tape.watch(x)
#     y = tf.nn.softmax(x)
#     loss = tf.reduce_sum(y*label)
#     grads = tape.gradient(loss, x)
#     print (grads)

# import tensorflow as tf

# x = np.random.normal(size=[5, 6])
# aa = Log()
# out = aa.forward(x) # shape(5, 4)
# grad = aa.backward(label)
# print (grad)

# with tf.GradientTape() as tape:
#     x= tf.constant(x)
#     tape.watch(x)
#     y = tf.math.log(x)
#     loss = tf.reduce_sum(y*label)
#     grads = tape.gradient(loss, x)
#     print (grads)

# Final Gradient Check

In [4]:
x = np.random.normal(size=[5, 6])
W1 = np.random.normal(size=[6, 5])
W2 = np.random.normal(size=[5, 6])
label = np.zeros_like(x)
label[0, 1] = 1.
label[1, 0] = 1
label[2, 3] = 1
label[3, 5] = 1
label[4, 0] = 1

x = torch.tensor(x, dtype=torch.float32, requires_grad=True)
W1 = torch.tensor(W1, dtype=torch.float32, requires_grad=True)
W2 = torch.tensor(W2, dtype=torch.float32, requires_grad=True)
label = torch.tensor(label, dtype=torch.float32)

mul_h1 = Matmul()
mul_h2 = Matmul()
relu = Relu()
softmax = Softmax()
log = Log()

h1 = mul_h1.forward(x.detach().numpy(), W1.detach().numpy())
h1_relu = relu.forward(h1)
h2 = mul_h2.forward(h1_relu, W2.detach().numpy())
h2_soft = softmax.forward(h2)
h2_log = log.forward(h2_soft)

h2_log_grad = log.backward(label.detach().numpy())
h2_soft_grad = softmax.backward(h2_log_grad)
h2_grad, W2_grad = mul_h2.backward(h2_soft_grad)
h1_relu_grad = relu.backward(h2_grad)
h1_grad, W1_grad = mul_h1.backward(h1_relu_grad)

print(W1_grad)
print('--' * 20)

h1 = torch.matmul(x, W1)
h1_relu = torch.relu(h1)
h2 = torch.matmul(h1_relu, W2)
prob = torch.softmax(h2, dim=-1)
log_prob = torch.log(prob)
loss = torch.sum(label * log_prob)
loss.backward()
print(W1.grad)

[[-1.0812038   0.8077851  -1.9729984   0.          0.5861937 ]
 [ 0.18299548  0.33581474 -0.18907534  0.          0.29552728]
 [-1.8663749  -0.5637673  -1.4388145   0.         -0.35683972]
 [-0.19364344 -6.817115   -0.21657829  0.         -2.275575  ]
 [ 0.1536798  -6.1336737  -1.5475589   0.         -0.87423146]
 [-1.3478748  -2.552765   -1.8614624   0.         -0.5149436 ]]
----------------------------------------
tensor([[-1.0812,  0.8078, -1.9730,  0.0000,  0.5862],
        [ 0.1830,  0.3358, -0.1891,  0.0000,  0.2955],
        [-1.8664, -0.5638, -1.4388,  0.0000, -0.3568],
        [-0.1936, -6.8171, -0.2166,  0.0000, -2.2756],
        [ 0.1537, -6.1337, -1.5476,  0.0000, -0.8742],
        [-1.3479, -2.5528, -1.8615,  0.0000, -0.5149]])


## 建立模型

In [5]:
class myModel:
    def __init__(self):
        
        self.W1 = np.random.normal(size=[28*28+1, 100])
        self.W2 = np.random.normal(size=[100, 10])
        
        self.mul_h1 = Matmul()
        self.mul_h2 = Matmul()
        self.relu = Relu()
        self.softmax = Softmax()
        self.log = Log()
        
        
    def forward(self, x):
        x = x.reshape(-1, 28*28)
        bias = np.ones(shape=[x.shape[0], 1])
        x = np.concatenate([x, bias], axis=1)
        
        self.h1 = self.mul_h1.forward(x, self.W1) # shape(5, 4)
        self.h1_relu = self.relu.forward(self.h1)
        self.h2 = self.mul_h2.forward(self.h1_relu, self.W2)
        self.h2_soft = self.softmax.forward(self.h2)
        self.h2_log = self.log.forward(self.h2_soft)
            
    def backward(self, label):
        self.h2_log_grad = self.log.backward(-label)
        self.h2_soft_grad = self.softmax.backward(self.h2_log_grad)
        self.h2_grad, self.W2_grad = self.mul_h2.backward(self.h2_soft_grad)
        self.h1_relu_grad = self.relu.backward(self.h2_grad)
        self.h1_grad, self.W1_grad = self.mul_h1.backward(self.h1_relu_grad)
        
model = myModel()


## 计算 loss

In [6]:
def compute_loss(log_prob, labels):
     return np.mean(np.sum(-log_prob*labels, axis=1))
    

def compute_accuracy(log_prob, labels):
    predictions = np.argmax(log_prob, axis=1)
    truth = np.argmax(labels, axis=1)
    return np.mean(predictions==truth)

def train_one_step(model, x, y):
    model.forward(x)
    model.backward(y)
    model.W1 -= 1e-5* model.W1_grad
    model.W2 -= 1e-5* model.W2_grad
    loss = compute_loss(model.h2_log, y)
    accuracy = compute_accuracy(model.h2_log, y)
    return loss, accuracy

def test(model, x, y):
    model.forward(x)
    loss = compute_loss(model.h2_log, y)
    accuracy = compute_accuracy(model.h2_log, y)
    return loss, accuracy

## 实际训练

In [7]:
train_data, test_data = mnist_dataset()
train_label = np.zeros(shape=[train_data[0].shape[0], 10])
test_label = np.zeros(shape=[test_data[0].shape[0], 10])
train_label[np.arange(train_data[0].shape[0]), np.array(train_data[1])] = 1.
test_label[np.arange(test_data[0].shape[0]), np.array(test_data[1])] = 1.

for epoch in range(50):
    loss, accuracy = train_one_step(model, train_data[0], train_label)
    print('epoch', epoch, ': loss', loss, '; accuracy', accuracy)
loss, accuracy = test(model, test_data[0], test_label)

print('test loss', loss, '; accuracy', accuracy)

epoch 0 : loss 23.58157280271105 ; accuracy 0.09401666666666667
epoch 1 : loss 22.251923880109857 ; accuracy 0.13495
epoch 2 : loss 21.042085935221618 ; accuracy 0.16945
epoch 3 : loss 19.71694616900349 ; accuracy 0.21371666666666667
epoch 4 : loss 18.656690367726085 ; accuracy 0.2614166666666667
epoch 5 : loss 16.87259639450339 ; accuracy 0.3080833333333333
epoch 6 : loss 16.70137328728802 ; accuracy 0.33515
epoch 7 : loss 15.024991440614068 ; accuracy 0.36966666666666664
epoch 8 : loss 14.092425118988261 ; accuracy 0.41376666666666667
epoch 9 : loss 13.4050464825895 ; accuracy 0.43345
epoch 10 : loss 12.79736439533024 ; accuracy 0.4532
epoch 11 : loss 12.61603589434422 ; accuracy 0.46665
epoch 12 : loss 10.68304941813016 ; accuracy 0.5375
epoch 13 : loss 9.208498031403606 ; accuracy 0.5817
epoch 14 : loss 9.025780455021899 ; accuracy 0.6034333333333334
epoch 15 : loss 8.387182664468563 ; accuracy 0.6182166666666666
epoch 16 : loss 8.873965851330302 ; accuracy 0.6145166666666667
epoch